
# Atividade Named-Entity Recognition (NER)

### Desafio

Crie a rotina para processamento dos textos, preparando os documentos para respeitar a estrutura proposta para rodar uma rede neural capaz de realizar a atividade de reconhecimento de entidades.

Lembre que o resultado deverá ter três colunas (identificador, palavra e tag) como exemplificado na tabela abaixo:

| identificador | palavra | tag |
|:--:|:--:|:--:|
|102|tronco|O|
|102|da|O|
|102|coronaria|vaso|
|102|esquerda|vaso|
|102|com|O|
|102|trajeto|O|

Você deverá considerar as seguintes classes e termos por classe:

-   **vaso**:
	- descendente anterior;
	- coronaria direita;
	- coronaria esquerda;
	- circunflexa;
	- primeiro ramo marginal;
	- segundo ramo marginal;
	- terceiro ramo marginal;
	- primeiro ramo diagonal;
	- segundo ramo diagonal;
	- terceiro ramo diagonal;
	- ventricular posterior;
	- arteria diagonalis;
	- descendente posteiror.
-   **trajeto**:
	- trajeto intramiocardico;
	- origem;
	- retroaortico;
	- interaortopulmonar;
	- valsalva;
	- seio;
	- sinotubular.
-   **placa**:
	- placa;
	- placas;
	- ateromatose.
-   **composição**:
	- calcificada;
	- calcificadas;
	- densamente calcificada;
	- densamente calcificadas;
	- densa;
	- densamente calcificado;
	- parcialmente calcificada;
	- predominantemente calcificada;
	- predominantemente calcificadas;
	- mista;
	- mistas;
	- predominantemente nao calcificada;
	- predominantemente não calcificada;
	- parcialmente não calcificada;
	- principalmente calcificada;
	- principalmente não calcificada;
	- principalmente nao calcificada;
	- predominio calcificado;
	- predominio nao calcificado;
	- não calcificada.
-   **grau**:
	- sem redução luminal;
	- discreta;
	- irregularidades parietais;
	- irregularidade parietal;
	- menor que 50%;
	- irregularidades luminais;
	- irregularidade luminal;
	- irregularidade;
	- irregularidades;
	- proxima de 50%;
	- proximo de 50%;
	- entre 50 e 70%;
	- 50%;
	- acima de 50%;
	- maior que 50%;
	- ao redor de 50%;
	- cerca de 50%;
	- em torno de 50%;
	- acima de 70%;
	- 70%;
	- cerca de 70%;
	- ao redor de 70%;
	- em torno de 70%;
	- moderada;
	- moderada reducao luminal;
	- reducao luminal moderada;
	- estenose moderada;
	- grau pelo menos moderado;
	- grau moderado;
	- acentuada;
	- suboclusao;
	- reducao luminal critica;
	- estenose critica;
	- oclusao;
	- ocluido;
	- ocluida.
-   **modificador V**:
	- modificador V;
	- vulnerabilidade;
	- remodelamento positivo;
	- baixa atenuação;
	- napking ring;
	- anel de guardanapo;
	- spot calcifications;
	- remodelamento arterial positivo.
-   **stent**:
	- stent.
-   **redução stent**:
	- hiperplasia neointimal;
	- neointimal;
	- proliferação neointimal.
-   **enxerto**:
	- enxerto.

In [71]:
# Resolução
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_rows', None)

df = pd.read_excel('https://github.com/pgiaeinstein/ner-aula/raw/master/data_coronaria.xlsx')

In [78]:
df.shape

(439, 1)

In [59]:
df.head()

,texto
0,paciente com escore de calcio de zero fase com...
1,paciente com escore de calcio de zero fase com...
2,paciente com escore de calcio de zero fase com...
3,paciente com escore de calcio zero fase com co...
4,paciente com escore de calcio de zero fase com...


In [18]:
reg_vaso = re.compile(r"(descendente anterior|descendente posterior|coronaria (esquerda|direita)|circunflexa|ventricular posterior|arteria diagonalis|((primeiro|segundo|terceiro|quarto)(\se\s|\s))+(ramos?)?\s?(diagon(al|ais)?|margin(al|ais)?)?)")
reg_trajeto = re.compile(r"(trajeto intramiocardico|retroaórtico|interaortopulmonar|valssalva|sinotubular|seio)")
reg_placa = re.compile(r"(placas?|ateromatose)")
reg_desc_placa = re.compile(r"((predominantemente nao |nao |parcialmente |parcialmente nao |predominantemente |principalmente |principalmente nao |densamente )?calcificadas?|mistas?|predominio calcificado|predominio nao calcificado|densamente calcificado|densa)")
reg_oclusao = re.compile(r"(oclusao|suboclusao|acentuada|\b70%?\b|\b50%?\b|moderada\s?(reducao)?\s?(luminal)?|discreta\b|(irregularidades?\s?(pariet(ais|al)|lumin(al|ais))?)|menor que 50%?|maior que 50%?|entre 50%? e 70%?|acima de 50%?|ao redor de 50%?|cerca de 50%?|em torno de 50%?|acima de 70%?|cerca de 70%?|ao redor de 70%?|em torno de 70%?|reducao luminal critica|estenose critica|ocluid(o|a)|reducao luminal moderada|estenose moderada|grau pelo menos moderado|grau moderado|proxima de 50%?|proximo de 50%?)")
reg_modificador = re.compile(r"(modificador v|vulnerabilidade|remodelamento positivo|baixa atenuacao|napking ring|anel de guardanapo|spot calcifications|baixa atenuacao|remodelamento arterial positivo)")
reg_stent = re.compile(r"(stent)")
reg_red_stent = re.compile(r"(proliferacao neointimal|hiperplasia neointimal|hiperplasia|neointimal)")
reg_enxerto = re.compile(r"(enxerto)")

In [65]:
lista_documentos = df['texto'].tolist()

In [66]:
def encontra_tags(texto):
    
    lista_retorno = list()
    
    for item in reg_vaso.finditer(texto):
        lista_retorno.append({
            'classe'  : 'vaso',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
    
    for item in reg_trajeto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'trajeto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_desc_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'desc_placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_oclusao.finditer(texto):
        lista_retorno.append({
            'classe'  : 'oclusao',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_modificador.finditer(texto):
        lista_retorno.append({
            'classe'  : 'modificador',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_red_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'red_stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_enxerto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'enxerto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })

    return sorted(lista_retorno, key = lambda x : x['inicial'])

In [67]:
def parse_texto(texto, lista_tags, indice):
    
    lista_retorno = list()
    lista_tmp = list()
    
    tamanho_texto = len(texto)
    tamanho_lista = len(lista_tags)
    
    ultima_pos = 0
    
    for index, achado in enumerate(lista_tags):
        
        if achado['inicial'] != 0 and index == 0:
            lista_tmp.append((texto[0:achado['inicial']], 'O'))
            ultima_pos = achado['inicial']

        if ultima_pos != achado['inicial']:
            lista_tmp.append((texto[ultima_pos:achado['inicial']], 'O'))
            
        lista_tmp.append((texto[achado['inicial']:achado['final']], achado['classe']))
        ultima_pos = achado['final']
            
        if index + 1 == tamanho_lista:
            lista_tmp.append((texto[ultima_pos:], 'O'))
        
    for item in lista_tmp:
        tag_atual = item[1]
        for palavra in item[0].split(' '):
            if palavra == '':
                continue
            lista_retorno.append({
                'identificador' : indice,
                'palavra'       : palavra,
                'tag'           : tag_atual
            }) 
    
    return lista_retorno
        

In [68]:
index = 1
lista_resultante = list()

for documento in lista_documentos:
    lista_tags_documento = encontra_tags(documento)
    lista_resultante.extend(parse_texto(documento, lista_tags_documento, index))
    index += 1

In [69]:
df_para_nn = pd.DataFrame(lista_resultante)

In [73]:
df_para_nn.head(200)

,identificador,palavra,tag
0,1,paciente,O
1,1,com,O
2,1,escore,O
3,1,de,O
4,1,calcio,O
5,1,de,O
6,1,zero,O
7,1,fase,O
8,1,com,O
9,1,contraste,O


### Modelos

In [74]:
import pandas as pd
import numpy as np

# data = pd.read_excel("new_output.xlsx")

In [75]:
data = df_para_nn

In [ ]:
# data = data.fillna(method="ffill")

In [76]:
data.tail(10)

,identificador,palavra,tag
48523,439,do,O
48524,439,mesmo,O
48525,439,lado,O
48526,439,compativel,O
48527,439,com,O
48528,439,sequela,O
48529,439,de,O
48530,439,complexo,O
48531,439,granulomatoso,O
48532,439,primario,O


In [79]:
f'Total de documentos na base: {len(data.identificador.unique())}'

'Total de documentos na base: 439'

In [140]:
words = list(set(data["palavra"].values))
words.append("ENDPAD")
n_words = len(words)
n_words

638

In [96]:
tags = list(set(data["tag"].values))
n_tags = len(tags)
n_tags

7

In [97]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["palavra"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("identificador").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["identificador".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [98]:
getter = SentenceGetter(data)

In [99]:
sent = getter.get_next()

In [100]:
sentences = getter.sentences

In [146]:
max_len = 300
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [147]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [148]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [149]:
word2idx['ENDPAD']

637

In [150]:
word2idx

{'as': 0,
 'segmentar': 1,
 'heterogenea': 2,
 'trifurcando': 3,
 'a': 4,
 'percentil': 5,
 '275': 6,
 '15': 7,
 'sua': 8,
 'variante': 9,
 'normalidade': 10,
 'angulada': 11,
 '13': 12,
 'inferior': 13,
 'quinto': 14,
 'tenue': 15,
 'lado': 16,
 'nos': 17,
 'bastante': 18,
 '168': 19,
 'focal': 20,
 '37': 21,
 'data': 22,
 'subramos': 23,
 'residuais': 24,
 'nodulo': 25,
 'avaliadas': 26,
 'expressao': 27,
 'padrao': 28,
 'ectasia': 29,
 'pervio': 30,
 'movimento': 31,
 'superficie': 32,
 'percentis': 33,
 'diafragmatico': 34,
 'unico': 35,
 'foram': 36,
 'descendente': 37,
 'ramificados': 38,
 'mas': 39,
 'todo': 40,
 'inespecificos': 41,
 '280': 42,
 'tortuoso': 43,
 'atenuacao': 44,
 'escore': 45,
 'avaliacao': 46,
 'areas': 47,
 'diminutas': 48,
 'pleural': 49,
 '47': 50,
 'porcao': 51,
 'esofagico': 52,
 'ostio': 53,
 'apenas': 54,
 'cisto': 55,
 'anteromedial': 56,
 'em': 57,
 'nas': 58,
 'variacao': 59,
 'assumindo': 60,
 'anomalo': 61,
 'evidencias': 62,
 'associado': 63,
 'ba

In [151]:
X[0]

array([628, 253,  45, 571, 258, 571, 563, 582, 253, 496, 592, 609, 194,
       267, 540, 592, 609, 253, 466, 428, 597, 174, 315,  57,  37,  84,
       428, 433, 587,  37,  84, 571, 134, 197, 253, 466, 428, 597, 174,
       337, 366, 428, 427, 584, 314, 571, 434, 597, 406, 256, 572, 587,
       433, 253, 466, 428, 597, 174, 337, 475, 159, 571, 134, 597, 406,
       256, 572, 366, 475, 159, 571, 297, 597, 406, 256, 572, 627,  37,
       458, 428, 452, 458, 253, 466, 428, 597, 174, 592, 236, 375,  54,
       562, 536, 604, 253, 466, 428, 597, 174, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 63

In [152]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [153]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [154]:
lista_documentos[1]

'paciente com escore de calcio de zero fase com contraste coronaria direita dominante tronco da coronaria esquerda com trajeto e calibre normais bifurcando em descendente anterior e circunflexa arteria descendente anterior com trajeto e calibre normais primeiro e segundo ramos diagonais de pequeno calibre ambos com discretas irregularidades parietais proximais arteria circunflexa com trajeto e calibre normais primeiro ramo marginal de pequeno calibre sem reducao luminal segundo ramo marginal de grande calibre sem reducao luminal coronaria direita com trajeto e calibre normais arterias descendente posterior e ventricular posterior com trajeto e calibre normais'

In [155]:
y[1]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2,
       2, 2, 2, 3, 3, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], d

In [156]:
tag2idx

{'vaso': 0,
 'modificador': 1,
 'O': 2,
 'oclusao': 3,
 'trajeto': 4,
 'desc_placa': 5,
 'placa': 6}

In [157]:
from tensorflow.keras.utils import to_categorical

In [158]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [159]:
from sklearn.model_selection import train_test_split

In [160]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=6)

### BI LSTM

In [161]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [162]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=32, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

In [163]:
model = Model(input, out)

In [164]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [165]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

Train on 355 samples, validate on 40 samples
Epoch 1/15
355/355 [==============================] - 3s 9ms/sample - loss: 1.0636 - acc: 0.8174 - val_loss: 0.3911 - val_acc: 0.8999
Epoch 2/15
355/355 [==============================] - 3s 8ms/sample - loss: 0.3649 - acc: 0.8955 - val_loss: 0.2948 - val_acc: 0.8999
Epoch 3/15
355/355 [==============================] - 3s 7ms/sample - loss: 0.3088 - acc: 0.8955 - val_loss: 0.2813 - val_acc: 0.8999
Epoch 4/15
355/355 [==============================] - 3s 8ms/sample - loss: 0.2935 - acc: 0.8955 - val_loss: 0.2691 - val_acc: 0.8999
Epoch 5/15
355/355 [==============================] - 3s 8ms/sample - loss: 0.2852 - acc: 0.8955 - val_loss: 0.2593 - val_acc: 0.8999
Epoch 6/15
355/355 [==============================] - 3s 8ms/sample - loss: 0.2694 - acc: 0.8955 - val_loss: 0.2570 - val_acc: 0.8999
Epoch 7/15
355/355 [==============================] - 3s 8ms/sample - loss: 0.2497 - acc: 0.8959 - val_loss: 0.2169 - val_acc: 0.9095
Epoch 8/15
355/35

In [167]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [168]:
test_pred = model.predict(X_te, verbose=1)

44/44 [==============================] - 0s 5ms/sample


In [169]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [170]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

ACC      : 98.4015%
PRECISION: 77.3756%
RECALL   : 76.3393%
F1       : 76.8539%


In [171]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

    oclusao       0.47      0.14      0.22        49
    trajeto       0.00      0.00      0.00        20
       vaso       0.79      0.94      0.86       534
      placa       0.00      0.00      0.00        34
 desc_placa       0.27      0.12      0.16        34
modificador       0.00      0.00      0.00         1

  micro avg       0.77      0.76      0.77       672
  macro avg       0.68      0.76      0.71       672



### BI-LSTM-CRF

In [172]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [173]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=32, input_length=max_len, mask_zero=True)(input)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
# model = TimeDistributed(Dense(64, activation="relu"))(model)
crf = CRF(n_tags)
out = crf(model)

AttributeError: in converted code:
    relative to /usr/local/lib/python3.7/dist-packages:

    tf_keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:292 call
        test_output = self.viterbi_decoding(X, mask)
    tf_keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:564 viterbi_decoding
        argmin_tables = self.recursion(input_energy, mask, return_logZ=False)
    tf_keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:527 recursion
        unroll=self.unroll)
    tensorflow/python/keras/backend.py:3732 rnn
        input_time_zero, tuple(initial_states) + tuple(constants))
    tf_keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:521 _step
        return self.step(input_energy_i, states, return_logZ)
    tf_keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:463 step
        m = K.slice(states[3], [0, t], [-1, 2])
    tensorflow/python/util/deprecation_wrapper.py:106 __getattr__
        attr = getattr(self._dw_wrapped_module, name)

    AttributeEr

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
print(classification_report(test_labels, pred_labels))